- To show GUI in Jupyter lab, need to install additional package.  
  See https://qiita.com/hyt-sasaki/items/2a940cd7387c408988be.

```bash
jupyter labextension install @jupyter-widgets/jupyterlab-manager
```

- If you use JupyterNotebook run below.

```bash
jupyter nbextension enable --py widgetsnbextension
```

In [4]:
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

from CRF import crf
from ipywidgets import interact,IntSlider,FloatSlider,BoundedIntText

### Get data

X: RGB image  
Y: Binary segmentation mask

In [6]:
X = np.array([np.array(Image.open(img_path).convert('RGB')) for img_path in sorted(glob.glob("data/original/*"))])
Y = np.array([np.array(Image.open(img_path).convert('L')) for img_path in sorted(glob.glob("data/binary/*"))])

Y = np.where(Y<=0.5,0,255)
X.shape,Y.shape

((599, 256, 256, 3), (599, 256, 256))

### Make GUI

In [15]:
gt_prob_slider = FloatSlider(value=0.8, min=0.1, max=1, step=0.05, description='gt_prob:')
gau_sxy_slider = IntSlider(value=3, min=1, max=100, step=1, description='gau_sxy:')
gau_compat_slider = IntSlider(value=3, min=1, max=10, step=1, description='gau_compat:')
bil_sxy_slider = IntSlider(value=80, min=1, max=100, step=1, description='bil_sxy:')
bil_srgb_slider = IntSlider(value=13, min=1, max=25, step=1, description='bil_srgb:')
bil_compat_slider = IntSlider(value=10, min=1, max=20, step=1, description='bil_compat:')
n_inference_slider = IntSlider(value=5, min=0, max=10, step=1, description='n_inference:')
image_number_text = BoundedIntText(value=0, min=0, max=len(X)-1, step=1, description='img num:',)

In [13]:
def show_result(x,y,crfimage):
    
    plt.figure(figsize=(14,10))
    plt.subplot(131)
    plt.imshow(x)
    plt.title("origin")
    plt.axis("off")
    
    plt.subplot(132)
    plt.imshow(y,cmap="gray")
    plt.title("before crf")
    plt.axis("off")
    
    plt.subplot(133)
    plt.imshow(crfimage)
    plt.title("after crf")
    plt.axis("off")
    plt.show()
    
    return


def interactive_CRF(image_number, gt_prob, n_inference, gau_sxy, gau_compat, bil_sxy, bil_srgb, bil_compat):
    
    x=X[image_number]
    y=Y[image_number]
    
    try:
        crfimage = crf(x,y,gt_prob, n_inference, gau_sxy, gau_compat, bil_sxy, bil_srgb, bil_compat) # return a RGB image
        # return a RGB image
    except: 
        crfimage = np.zeros(y.shape, dtype=np.uint8)

    show_result(x,y,crfimage)
    
    return 

### Activate interactive function

In [17]:
interact(interactive_CRF,
         image_number = image_number_text,
         gt_prob=gt_prob_slider, 
         n_inference=n_inference_slider,
         gau_sxy=gau_sxy_slider, 
         gau_compat=gau_compat_slider, 
         bil_sxy=bil_sxy_slider, 
         bil_srgb=bil_srgb_slider, 
         bil_compat=bil_compat_slider,
         )

interactive(children=(BoundedIntText(value=0, description='img num:', max=598), FloatSlider(value=0.8, descrip…

<function __main__.interactive_CRF(image_number, gt_prob, n_inference, gau_sxy, gau_compat, bil_sxy, bil_srgb, bil_compat)>